# Spam Ham Project Using Word2vec and AvgWord2vec

## Import Tools

In [3]:
!pip install --upgrade numpy
!pip install --upgrade gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 69.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import gensim
from gensim.models import Word2Vec , KeyedVectors
print(gensim.__version__)

4.3.3


In [3]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')
vec_king = wv['king']

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [5]:
len(vec_king)

300

## Getting Data

In [6]:
messages = pd.read_csv('/content/SMSSpamCollection.tsv' , sep = '\t' , names = ['label' , 'message'])

In [7]:
messages.head(10)

,label,message
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
5,ham,As per your request 'Melle Melle (Oru Minnamin...
6,spam,WINNER!! As a valued network customer you have...
7,spam,Had your mobile 11 months or more? U R entitle...
8,ham,I'm gonna be home soon and i don't want to tal...
9,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [8]:
messages.shape

(5568, 2)

## Data Cleaning

* Stemming
* Lemmitization
* StopWords

In [9]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
lemmatizer = WordNetLemmatizer()

In [11]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [17]:
len(corpus)

5568

In [12]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [13]:
words = []
for sentence in corpus:
    sent_token = sent_tokenize(sentence)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [69]:
corpus[1]

'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply'

In [70]:
words[1]

['free',
 'entry',
 'wkly',
 'comp',
 'win',
 'fa',
 'cup',
 'final',
 'tkts',
 'st',
 'may',
 'text',
 'fa',
 'receive',
 'entry',
 'question',
 'std',
 'txt',
 'rate',
 'apply']

In [16]:
len(words)

5560

In [41]:
words[1]

['free',
 'entry',
 'wkly',
 'comp',
 'win',
 'fa',
 'cup',
 'final',
 'tkts',
 'st',
 'may',
 'text',
 'fa',
 'receive',
 'entry',
 'question',
 'std',
 'txt',
 'rate',
 'apply']

## Let's Train Word2Vec From Scratch

In [71]:
model = gensim.models.Word2Vec(words)

In [73]:
# Number of all the vocabulary
len(model.wv.index_to_key)

1602

In [74]:
model.corpus_count

5560

In [75]:
model.epochs

5

In [76]:
model.wv.similar_by_word('good')

[('give', 0.9996325373649597),
 ('keep', 0.9996052384376526),
 ('day', 0.9995936751365662),
 ('much', 0.9995865821838379),
 ('thing', 0.9995748400688171),
 ('got', 0.9995703101158142),
 ('last', 0.9995630979537964),
 ('tell', 0.999545693397522),
 ('like', 0.9995417594909668),
 ('amp', 0.999541699886322)]

In [77]:
model.wv['good'].shape

(100,)

In [78]:
model.wv['good']

array([-2.07613602e-01,  4.14051354e-01,  8.65319893e-02,  1.57042831e-01,
        1.05786659e-01, -6.84962213e-01,  7.54091516e-02,  8.71598780e-01,
       -3.09369147e-01, -1.22072592e-01, -3.03988248e-01, -7.15848327e-01,
        1.25652617e-02,  5.96393272e-02,  1.11181125e-01, -3.13631773e-01,
        4.00257595e-02, -5.00960410e-01, -6.81904629e-02, -7.54671812e-01,
        2.07085311e-01,  2.60249138e-01,  8.26918259e-02, -2.52773821e-01,
       -2.00792149e-01,  6.02371432e-02, -3.75568688e-01, -3.50404084e-01,
       -4.35471565e-01,  4.44172546e-02,  5.05243301e-01,  4.05680016e-02,
        1.23642348e-01, -2.29287148e-01, -1.66442305e-01,  4.60666299e-01,
        1.47554383e-01, -5.28649986e-01, -2.58284867e-01, -8.28507483e-01,
        5.87718897e-02, -3.93313289e-01, -9.43515897e-02,  9.36245471e-02,
        3.31203282e-01, -1.38770923e-01, -2.33051270e-01, -1.34267703e-01,
        2.47841030e-01,  3.33758175e-01,  1.97707891e-01, -3.29445064e-01,
       -1.19652236e-02,  

In [79]:
len(model.wv['good'])

100

In [80]:
len(words)

5560

## Apply AvgWord2Vc
for find value of entire sentence not just one words

In [81]:
def av_word2vec(doc):
  # Calculate average word vector for the document
  return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key])

  # # Handle cases where no word vectors are found
  # if len(word_vectors) == 0:
  #   # Return a zero vector with the correct shape (100,)
  #   return np.zeros(model.vector_size)
  # else:
  #   # Calculate and return the average word vector
  #   return np.mean(word_vectors, axis=0)

In [82]:
from tqdm import tqdm

In [83]:
# Apply av_word2vec for entire sentences
X = []
for i in tqdm(range(len(words))):
    #print("hello", i) # Optional: comment out to reduce output
    X.append(av_word2vec(words[i]))


  0%|          | 0/5560 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5560/5560 [00:00<00:00, 8882.13it/s]


In [85]:
len(X)

5560

In [35]:
# Independent Features
X_new = np.array(X)

In [62]:
X_new.shape

(5560,)

In [54]:
len(corpus)

5568

In [55]:
messages.shape

(5568, 2)

In [37]:
X_new.shape

(5560,)

In [38]:
X_new[0].shape

()

In [39]:
X_new[0]

-0.0007571654859930277

In [48]:
# Dependent Features or Outpu Features
y=pd.get_dummies(messages['label'])
y=y.iloc[:,1].values

In [58]:
# index in corpus array that empthy
empty_indices = [i for i, x in enumerate(corpus) if len(x) == 0]
len(empty_indices)

8

we have 8 records after procssing text give us a empthy case inside this records in corpus array , so we lost 8 records

In [61]:
# filtering by chosing just index that conatining text from empthy one
y_filter=messages[list(map(lambda x: len(x)>0 , corpus))]
y=pd.get_dummies(y_filter['label'])
y=y.iloc[:,0].values

In [60]:
y_filter

,label,message
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
...,...,...
5563,spam,This is the 2nd time we have tried 2 contact u...
5564,ham,Will ü b going to esplanade fr home?
5565,ham,"Pity, * was in mood for that. So...any other s..."
5566,ham,The guy did some bitching but I acted like i'd...


In [89]:
# This is the final independent features
df = pd.DataFrame()
for i in range(0 ,len(X)):
  df = pd.concat([df, pd.DataFrame(X[i].reshape(1,-1))], ignore_index=True) # Use pd.concat instead of append

<ipython-input-89-c742d35278d0>:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(X[i].reshape(1,-1))], ignore_index=True) # Use pd.concat instead of append


In [90]:
df.shape

(5560, 1)

In [91]:
df.head()

,0
0,-0.001229
1,-0.001825
2,-0.001885
3,-0.002486
4,-0.000593


In [92]:
# Independent Features
X = df

## Split Data into train and test Data Set

In [93]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [94]:
X_train.head()

,0
1826,-0.002137
3724,-0.002680
624,-0.002629
2250,-0.001054
500,-0.002165


## Modeling

In [96]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 1000, criterion = 'entropy')
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=1000)

In [97]:
y_preds = classifier.predict(X_test)

In [134]:
dt = pd.DataFrame({'Actual': y_test, 'Predicted': y_preds})
dt.Predicted.value_counts()

,count
Predicted,
True,937
False,175


In [98]:
from sklearn.metrics import confusion_matrix , classification_report , recall_score , precision_score
print(classification_report(y_test , y_preds))

              precision    recall  f1-score   support

       False       0.43      0.49      0.46       154
        True       0.92      0.90      0.91       958

    accuracy                           0.84      1112
   macro avg       0.67      0.69      0.68      1112
weighted avg       0.85      0.84      0.84      1112



In [100]:
print("recall : ",recall_score(y_test , y_preds))
print("precision : ",precision_score(y_test , y_preds))

recall :  0.8956158663883089
precision :  0.9156883671291356


## Predict on some new data

In [102]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5568 entries, 0 to 5567
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5568 non-null   object
 1   message  5568 non-null   object
dtypes: object(2)
memory usage: 87.1+ KB


In [103]:
test_data = "Congratulations! You’ve won a $1000 Walmart gift card. Click here to claim now."

In [104]:
new_corpus =[]
review = re.sub('[^a-zA-Z]', ' ', test_data)
review = review.lower()
review = review.split()
review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
review = ' '.join(review)
new_corpus.append(review)

In [105]:
new_corpus

['congratulation walmart gift card click claim']

In [106]:
words = []
for sentence in new_corpus:
    sent_token = sent_tokenize(sentence)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [107]:
words

[['congratulation', 'walmart', 'gift', 'card', 'click', 'claim']]

In [109]:
def av_word2vec(doc):
  # Calculate average word vector for the document
  return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key])

In [110]:
# Apply av_word2vec for entire sentences
test_data = []
for i in tqdm(range(len(words))):
    #print("hello", i) # Optional: comment out to reduce output
    test_data.append(av_word2vec(words[i]))

100%|██████████| 1/1 [00:00<00:00, 828.42it/s]


In [112]:
test_data

[-0.0017282795]

In [114]:
# Reshape test_data to a 2D array
test_data = np.array(test_data).reshape(1, -1)

In [115]:
classifier.predict(test_data)

array([ True])

### Function that can predict text is a spam or not

In [119]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
from tqdm import tqdm
import numpy as np

def predict_spam(text, model, classifier, lemmatizer):
    """Predicts whether a given text is spam or not.

    Args:
        text: The input text to be classified.
        model: The trained Word2Vec model.
        classifier: The trained RandomForestClassifier.
        lemmatizer: The WordNetLemmatizer instance.

    Returns:
        0 if the text is predicted as ham (not spam), 1 if predicted as spam.
    """

    # 1. Preprocessing
    new_corpus = []
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    new_corpus.append(review)

    words = []
    for sentence in new_corpus:
        sent_token = sent_tokenize(sentence)
        for sent in sent_token:
            words.append(simple_preprocess(sent))

    # 2. Word Embedding using Word2Vec
    def av_word2vec(doc):
        # Calculate average word vector for the document, handling missing words
        word_vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
        if len(word_vectors) == 0:
            # Return a zero vector with the correct shape if no word vectors are found
            return np.zeros(model.vector_size)
        else:
            return np.mean(word_vectors, axis=0)

    test_data = []
    for i in range(len(words)):
        test_data.append(av_word2vec(words[i]))

    # 3. Reshape test_data based on the model's expected input shape
    test_data = np.array(test_data).reshape(1, -1)

    # If model was trained on a single feature, take the mean of all features:
    if classifier.n_features_in_ == 1:
        test_data = np.mean(test_data, axis=1).reshape(1, -1)

    # 3. Prediction using RandomForestClassifier
    prediction = classifier.predict(test_data)[0]
    return prediction  # Return 0 for ham, 1 for spam

In [139]:
text = 'Congratulations! You’ve won a $1000 Walmart gift card. Click here to claim now.'
text_2 = "Don’t forget to bring your laptop for tomorrow’s workshop"
text_3="hello everyone are you ready for meeting"
predction =predict_spam(text_3 , model , classifier , lemmatizer)
predction

True